In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from skimage import color
from skimage.feature import hog
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
from skimage.io import imread, imshow
from skimage.transform import resize
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split

In [ ]:
## Annotated Data to be downloaded from https://vision.cs.uiuc.edu/attributes/  

!tar -xvzf /content/attribute_data.tar.gz

attribute_data/
attribute_data/ayahoo_test.txt
attribute_data/README
attribute_data/attribute_names.txt
attribute_data/apascal_test.txt
attribute_data/read_att_data.m
attribute_data/apascal_train.txt
attribute_data/class_names.txt
attribute_data/display_images.m


In [ ]:
## Download images
!wget -c http://vision.cs.uiuc.edu/attributes/ayahoo_test_images.tar.gz
!tar -xvzf /content/ayahoo_test_images.tar.gz
# !wget -c http://pascallin.ecs.soton.ac.uk/challenges/VOC/voc2008/

In [ ]:
data = pd.read_csv('/content/annotated_yahoo.csv')

with open('/content/attribute_names.txt', 'r') as content: ## This is the 33 attribute names present in the affordanceKB paper
    lab = content.read()

top_33_attr = [line for line in lab.split('\n') if len(line) > 0]

data = data.rename(columns={'2D Boxy': 'Boxy_2D', '3D Boxy': 'Boxy_3D', 
                               'Vert Cyl': 'Vert_Cyl',
                               'Horiz Cyl':'Horiz_Cyl', 'Furn. Leg': 'Furn_Leg',
                               'Furn. Back': 'Furn_Back',
                               'Furn. Seat': 'Furn_Seat', 'Furn. Arm': 'Furn_Arm'})

data = data[['image'] + top_33_attr]
data.head()

,image,Boxy_2D,Boxy_3D,Round,Vert_Cyl,Horiz_Cyl,Tail,Head,Wheel,Pedal,...,Wood,Cloth,Furry,Glass,Feather,Wool,Clear,Shiny,Vegetation,Leather
0,donkey_1.jpg,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,donkey_1.jpg,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,donkey_100.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,donkey_102.jpg,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,donkey_111.jpg,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
import keras, tensorflow
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
# from keras.utils import load_img
from tensorflow.keras.utils import load_img, img_to_array
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

In [ ]:
train, test = train_test_split(data, random_state=42, test_size=0.1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
train.head()

,image,Boxy_2D,Boxy_3D,Round,Vert_Cyl,Horiz_Cyl,Tail,Head,Wheel,Pedal,...,Wood,Cloth,Furry,Glass,Feather,Wool,Clear,Shiny,Vegetation,Leather
0,bag_599.jpg,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,goat_212.jpg,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,building_819.jpg,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
3,jetski_10.jpg,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,bag_332.jpg,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
train_image = []
for i in tqdm(range(train.shape[0])):
    img = load_img('/content/ayahoo_test_images/'+train['image'][i],target_size=(200,200,3))
    img = img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)

100%|██████████| 2379/2379 [00:16<00:00, 148.09it/s]


In [ ]:
y = np.array(train.drop(['image'],axis=1))
y.shape

(2379, 33)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(200,200,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(33, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 196, 196, 16)      1216      
                                                                 
 max_pooling2d_22 (MaxPoolin  (None, 98, 98, 16)       0         
 g2D)                                                            
                                                                 
 dropout_30 (Dropout)        (None, 98, 98, 16)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 94, 94, 32)        12832     
                                                                 
 max_pooling2d_23 (MaxPoolin  (None, 47, 47, 32)       0         
 g2D)                                                            
                                                                 
 dropout_31 (Dropout)        (None, 47, 47, 32)       

In [ ]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:

model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/20
67/67 [==============================] - 161s 2s/step - loss: 0.4464 - accuracy: 0.0929 - val_loss: 0.3239 - val_accuracy: 0.1891
Epoch 2/20
67/67 [==============================] - 153s 2s/step - loss: 0.3065 - accuracy: 0.1182 - val_loss: 0.2615 - val_accuracy: 0.1849
Epoch 3/20
67/67 [==============================] - 158s 2s/step - loss: 0.2743 - accuracy: 0.1425 - val_loss: 0.2507 - val_accuracy: 0.1849
Epoch 4/20
67/67 [==============================] - 154s 2s/step - loss: 0.2608 - accuracy: 0.1728 - val_loss: 0.2402 - val_accuracy: 0.2101
Epoch 5/20
67/67 [==============================] - 152s 2s/step - loss: 0.2486 - accuracy: 0.1681 - val_loss: 0.2322 - val_accuracy: 0.2689
Epoch 6/20
67/67 [==============================] - 151s 2s/step - loss: 0.2416 - accuracy: 0.1966 - val_loss: 0.2258 - val_accuracy: 0.2563
Epoch 7/20
67/67 [==============================] - 152s 2s/step - loss: 0.2334 - accuracy: 0.2055 - val_loss: 0.2222 - val_accuracy: 0.2773
Epoch 8/20
67

## Predicting visual attributes for the noun phrases extracted earlier

In [ ]:
noun_phrases_images = pd.read_csv('/content/images.csv')

noun_phrases_images.head()

,Unnamed: 0,noun,img,img_id
0,0,Product and geography,https://cs.stanford.edu/people/rak248/VG_100K/...,713545
1,1,Product and geography,https://cs.stanford.edu/people/rak248/VG_100K/...,1159702
2,2,Product and geography,https://cs.stanford.edu/people/rak248/VG_100K/...,2320213
3,3,Product and geography,https://cs.stanford.edu/people/rak248/VG_100K/...,2321812
4,4,Product and geography,https://cs.stanford.edu/people/rak248/VG_100K_...,2417008


In [ ]:
noun_phrase_to_image_mapping = {} # add multiple images for each np 
for id, rows in noun_phrases_images.iterrows():
    nps = rows[1]
    img_link = rows[2]  
    if nps in noun_phrase_to_image_mapping.keys():
        noun_phrase_to_image_mapping[nps].append(img_link)
    else:
        noun_phrase_to_image_mapping[nps] = [img_link]

In [ ]:
from io import BytesIO
import urllib




def loadImage(URL):
    with urllib.request.urlopen(URL) as url:
        img = load_img(BytesIO(url.read()), target_size=(200,200,3))
        img = img_to_array(img)
        img = img/255
    return img


threshold = 0.4
im = noun_phrase_to_image_mapping['Product and geography']

pred_image = []
for i in tqdm(range(len(im))):
    print(im[i])
    img = loadImage(im[i])
    pred_image.append(img)
X_pred = np.array(pred_image)


classes = np.array(train.columns[2:])
# proba = model.predict(img.reshape(1,128,128,3))


proba = model.predict(X_pred)
proba = np.array([[0 if a_ < threshold else 1 for a_ in a] for a in proba])
top_3 = np.argsort(proba[0])

print(proba)
# top_3
# X_test.shape

  0%|          | 0/5 [00:00<?, ?it/s]

https://cs.stanford.edu/people/rak248/VG_100K/713545.jpg


 20%|██        | 1/5 [00:00<00:02,  1.41it/s]

https://cs.stanford.edu/people/rak248/VG_100K/1159702.jpg


 40%|████      | 2/5 [00:01<00:02,  1.45it/s]

...

https://cs.stanford.edu/people/rak248/VG_100K/2321812.jpg


 80%|████████  | 4/5 [00:02<00:00,  1.75it/s]

https://cs.stanford.edu/people/rak248/VG_100K_2/2417008.jpg


100%|██████████| 5/5 [00:02<00:00,  1.70it/s]


1/1 [==============================] - 0s 225ms/step
[[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [ ]:
classes = np.array(train.columns[1:])
classes

array(['Boxy_2D', 'Boxy_3D', 'Round', 'Vert_Cyl', 'Horiz_Cyl', 'Tail',
       'Head', 'Wheel', 'Pedal', 'Handlebars', 'Text', 'Label',
       'Furn_Leg', 'Furn_Back', 'Furn_Seat', 'Furn_Arm', 'Rein', 'Saddle',
       'Pot', 'Screen', 'Skin', 'Metal', 'Plastic', 'Wood', 'Cloth',
       'Furry', 'Glass', 'Feather', 'Wool', 'Clear', 'Shiny',
       'Vegetation', 'Leather'], dtype=object)

In [ ]:
from tqdm import tqdm

def most_freq(lst):
    return max(set(lst), key=lst.count)

df_dict = {}
df_dict['noun_phrase'] = []
final_attr_labels = {}
i = 0
for nps, links in tqdm(noun_phrase_to_image_mapping.items()):
    df_dict['noun_phrase'].append(nps)
    # final_attr_labels[nps] = {}
    pred_image = []
    for im in links:
        img = loadImage(im)
        pred_image.append(img)
    X_pred = np.array(pred_image)
    proba = model.predict(X_pred)
    proba = np.array([[0 if a_ < threshold else 1 for a_ in a] for a in proba])
    predictions = proba.sum(axis=0)
    predictions = np.array([1 if a_ > 2 else 0 for a_ in predictions])

    print(predictions)
    for category in classes:
        if category in final_attr_labels.keys():
            final_attr_labels[category].append(predictions[list(classes).index(category)])
        else:
            final_attr_labels[category] = [predictions[list(classes).index(category)]]

    
    # if i == 3:
    #     break
print(final_attr_labels)

  0%|          | 0/174 [00:00<?, ?it/s]

1/1 [==============================] - 0s 114ms/step


  1%|          | 1/174 [00:03<08:52,  3.08s/it]

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 [==============================] - 0s 127ms/step


  1%|          | 2/174 [00:05<08:10,  2.85s/it]

...

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 [==============================] - 0s 106ms/step


 99%|█████████▉| 173/174 [09:09<00:02,  2.74s/it]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 [==============================] - 0s 116ms/step


100%|██████████| 174/174 [09:12<00:00,  3.17s/it]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
{'Boxy_2D': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'Boxy_3D': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
for lab in top_33_attr:
    df_dict[lab] = final_attr_labels[lab]

df = pd.DataFrame(df_dict)

df.head(20)

,noun_phrase,Boxy_2D,Boxy_3D,Round,Vert_Cyl,Horiz_Cyl,Tail,Head,Wheel,Pedal,...,Wood,Cloth,Furry,Glass,Feather,Wool,Clear,Shiny,Vegetation,Leather
0,Product and geography,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,cream,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
2,work,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,You,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,the things,0,0,0,0,0,1,1,0,0,...,0,0,1,0,0,0,0,0,0,0
5,the following level,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
6,the people,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,A member of my team,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,A member,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,my team,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.to_csv("visual_attribute_predictions.csv")

In [ ]:
with open("/content/relation_mln.txt", 'r') as content: 
    mln_rules = content.readlines()[11:]




In [ ]:
def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

replace_dict = {'boxy 2d': 'boxy_2d', 'boxy 3d': 'boxy_3d', 
                               'vertical cylindrical': 'vert_cyl',
                               'vertical cylindrical':'horiz_cyl', 
                               'furniture leg': 'furn_leg',
                               'furniture Back': 'furn_back',
                               'furniture Seat': 'furn_seat', 
                               'furniture Arm': 'furn_arm'}

# replace_all(mln_rules, replace_dict)

mln_rules = [x.strip('\n') for x in mln_rules]
mln_rules = [replace_all(x, replace_dict) for x in mln_rules]
mln_scores = [x.split()[0] for x in mln_rules]
mln_attr = [x.split()[1] for x in mln_rules]
mln_affordance = [x.split()[-1] for x in mln_rules]

print(mln_scores[0], mln_attr[0], mln_affordance[0])

-0.0717154 boxy_2d grasp


In [ ]:
objects = 'The tennis shoes'
# objects = 'my team'
# objects = 'that on television'


selected_df = df.loc[df['noun_phrase']==objects]

for category in top_33_attr:
    if selected_df[category].values[0] == 1:
        if category.lower() in mln_attr:
            indexes = mln_attr.index(category.lower())
            print(category, mln_affordance[indexes], mln_scores[indexes])

# selected_df['Boxy_2D'].values[0]


Head grasp -0.023025
Furry grasp -0.0907226
